# Preprocessing on Amazon's Fine Food Analysis

### Amazon's Fine Food reviews dataset :  https://www.kaggle.com/snap/amazon-fine-food-reviews

The Amazon Fine Food Reviews dataset consists of reviews of fine foods from Amazon.
   - Number of reviews: 568,454
   - Number of users: 256,059
   - Number of products: 74,258
   - Timespan: Oct 1999 - Oct 2012
   - Number of Attributes/Columns in data: 10(including class attribute)

Attribute Information:

   - Id
   - ProductId - unique identifier for the product
   - UserId - unqiue identifier for the user
   - ProfileName
   - HelpfulnessNumerator - number of users who found the review helpful
   - HelpfulnessDenominator - Total number of users who indicated whether they found the review helpful or not
   - Time - timestamp for the review
   - Summary - brief summary of the review
   - Text - text of the review
   - Score(Class Label) - rating between 1 and 5 (rating 1 & 2 is negative, rating 4 & 5 is positive and rating 3 is neutral)

## Objective
   - Complete a data cleaning and data preprocessing and store it in a separate database, for future use.

###  Loading required libraries

In [1]:
import warnings
from tqdm import tqdm_notebook as tqdm
import re, os, sqlite3,pickle
import pandas as pd 
import nltk
from nltk.stem import SnowballStemmer as snow
from nltk.corpus import stopwords

In [2]:
warnings.simplefilter('ignore')

Since the class-label 'Score' contains 5 different value including 3 which is neither positive nor negative review. For this study we omit 3 star rating and change the rest to positive(4 & 5 star) as 1 or negative(1 & 2 star) as 0  


### Changing class label to string

In [3]:
link = sqlite3.connect('../database.sqlite')
con_data= pd.read_sql_query(''' SELECT * FROM REVIEWS WHERE SCORE != 3''',link)
print(con_data.shape)
link.close()

(525814, 10)


In [4]:
con_data.head(3)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [5]:
d=con_data['Score']
con_data['Score'] = list(map(lambda x: 1 if x>3 else 0,d ))    # changing the Score values to 1(positive) \
con_data['Score'].shape,con_data.shape                         # and 0(negative)


((525814,), (525814, 10))

In [6]:
con_data.head(3),con_data.Time.min()

(   Id   ProductId          UserId                      ProfileName  \
 0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
 1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
 2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
 
    HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
 0                     1                       1      1  1303862400   
 1                     0                       0      0  1346976000   
 2                     1                       1      1  1219017600   
 
                  Summary                                               Text  
 0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
 1      Not as Advertised  Product arrived labeled as Jumbo Salted Peanut...  
 2  "Delight" says it all  This is a confection that has been around a fe...  ,
 939340800)

## Data cleaning

In [7]:
sorted_data = con_data.sort_values('ProductId', axis=0, ascending=True, 
                                   inplace=False, kind='quicksort', na_position='last')  # Deduplication
final_data = sorted_data.drop_duplicates(subset={'UserId','ProfileName','Time','Text'}, 
                                         keep='first', inplace=False)
sorted_data.shape,final_data.shape

((525814, 10), (364173, 10))

In [8]:
final_data = final_data[final_data['HelpfulnessDenominator'] >= final_data['HelpfulnessNumerator']]

##### ProductId's B00004CI84, B00004CXX9 - belongs to the movie called 'Beetle Juice'  

In [9]:
final = final_data[final_data['ProductId'] != 'B00004CXX9' ]  
cleaned_data = final[final['ProductId'] != 'B00004CI84' ]
cleaned_data.shape

(363980, 10)

In [10]:
a,b=cleaned_data.Score.value_counts()
print('%d positive reviews and %d negative reviews'%(a,b) )

306886 positive reviews and 57094 negative reviews


 Conclusion:
   - The dataset is highly imbalanced 

## Data preprocessing

In [11]:
cleaned_data.head(3)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,1,939340800,EVERY book is educational,this witty little book makes my son laugh at l...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,1,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc..."
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,1,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...


In [12]:
stop = set(stopwords.words('english'))    #initializing stopwords as a set
print(stop)       
len(stop)


{"haven't", 'weren', 'herself', 'himself', 'he', 'each', 'own', 'won', 'on', 'doesn', 'no', "aren't", 'am', 'once', 'in', 'too', 'not', 'mightn', 'other', 'hers', "should've", 'those', 't', 'myself', 'all', 'some', 'hadn', 'is', 're', 'did', 'then', 'from', 's', 'where', 'now', "doesn't", 'who', 'are', 'our', 'themselves', 'ain', 'don', 'the', 'as', 'mustn', 'if', "hasn't", 'below', 'out', 'will', 'has', 'wasn', 'o', 'these', 'haven', 'shouldn', 'it', 'whom', 'both', 've', "needn't", 'again', 'm', 'couldn', 'until', 'further', "couldn't", 'a', "it's", 'more', 'having', 'do', "mightn't", 'this', 'while', "shan't", 'yourselves', 'needn', 'only', 'didn', 'after', 'between', 'isn', 'about', 'against', 'does', 'there', 'his', "wasn't", 'theirs', 'doing', 'before', 'ma', 'an', 'and', 'ours', 'can', 'into', 'why', "hadn't", "that'll", "shouldn't", 'being', 'their', 'same', "weren't", 'ourselves', 'was', 'my', "isn't", 'under', 'we', 'wouldn', 'to', "wouldn't", 'been', 'up', 'few', "she's", 'n

179

###### Removing all the negative words from stopword set and Storing it in seperate set called nstop

In [13]:
nstop={'no','not','nor'}
for el in stop:
    if ("n't" in el):
        nstop.add(el)

In [14]:
stop = stop - nstop 
len(stop)                                    #stop = stopwords without negative words  

158

In [15]:
def cleanhtml(x):
    c = re.compile('<.*?>')                 
            # Removing all the html tags if present and replacing with 'space'
    cleaned = re.sub(c,' ', x)
    return cleaned

def cleanpun(x):
    c = re.sub(r'[?|!|\'|"|#]',r'',x)        
            # removing single quotes without replacing any character \ 
            #  is essential in  preserving sentiment words
    cleaned = re.sub(r'[.|,|)|(|\|/|\n]',r' ',c)
    return  cleaned.lower()                  
            # returning with lower case

sno =snow('english')                        
            #initializing snowballstemmer as sno

In [16]:
n=set(map(cleanpun,nstop))
print(n,sep='\t')

{'wouldnt', 'shant', 'shouldnt', 'hadnt', 'neednt', 'no', 'mightnt', 'werent', 'wont', 'doesnt', 'not', 'nor', 'isnt', 'dont', 'couldnt', 'arent', 'wasnt', 'mustnt', 'didnt', 'hasnt', 'havent'}


In [17]:
with open('./nstop.pickle', 'wb') as f:
    pickle.dump(n, f)

In [19]:
def cleaning_data(clean_data,stem1):
    """
    Performing data cleaning and stemming if necessary.
    """
    i=0
    words = ' '
    empty1 = []
    
    for sent in tqdm(clean_data):   
                    #tqdm for progress bar
        e2=[]   
        sentence = cleanhtml(sent)                 
                    # cleaning html tags
        sentence = cleanpun(sentence)              
                    # cleaning all punctuation and returning all lowercase
        for word in sentence.split():             
                    # getting a word in sentence separated by space 
            
            if ((len(word) > 2) & (word.isalpha() ) & (word not in stop )): 
                    # checking length, authenticity and if the given word is a stopword
                if(stem1):    
                    s= sno.stem(word).encode('utf8')   
                        # snowball stemming 
                else:
                    s=word.encode('utf8')
                e2.append(s)
                 
            else:
                continue
            
            words = b' '.join(e2)     
                    # joinning all the words by removing 'b' to form the sentence structure.
        i= i+1  
        empty1.append(words)                 
                    # list of sll processed sentence
    return empty1

#### preprocessing review 'Text' feature with stemming and storing it as 'c_text'

In [20]:
cleaned_data['c_text']= cleaning_data(
                        clean_data=cleaned_data['Text'].values,
                        stem1=True)                  
     
cleaned_data['c_text']=cleaned_data['c_text'].str.decode('utf-8')

#### preprocessing review 'Summary' feature with stemming and storing it as 'c_summary'

In [21]:
cleaned_data['c_summary']= cleaning_data(
                        clean_data=cleaned_data['Summary'].values,
                        stem1=True)                  

cleaned_data['c_summary']=cleaned_data['c_summary'].str.decode('utf-8')
  

#### preprocessing review 'Text' feature without stemming and storing it as 'nostem_text'

In [22]:
cleaned_data['nostem_text']= cleaning_data(
                            clean_data=cleaned_data['Text'].values,
                            stem1=False)                  

cleaned_data['nostem_text']=cleaned_data['nostem_text'].str.decode('utf-8')

####  preprocessing review 'Summary' feature without stemming and storing it as 'nostem_text'

In [23]:
cleaned_data['nostem_summary']= cleaning_data(
                        clean_data=cleaned_data['Summary'].values,
                        stem1=False)                  
           
cleaned_data['nostem_summary']=cleaned_data['nostem_summary'].str.decode('utf-8')

In [24]:
cleaned_data.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text', 'c_text',
       'c_summary', 'nostem_text', 'nostem_summary'],
      dtype='object')

In [25]:
print(cleaned_data.shape)
cleaned_data.Time.min()

(363980, 14)


939340800

In [26]:
link = sqlite3.connect('cleaned_reviews.sqlite')        
c=link.cursor()
link.text_factory = str
cleaned_data.to_sql('creviews', link, if_exists='replace', index=False)
link.close()                                 
            #saving all the data in a file called "cleaned_reviews.sqlite"

## Conclusion:
   - Amazon fine food review data has been cleaned and stored in sql database called 'cleaned_reviews.sqlite'.